CREAR LA DATA BASE

In [ ]:
import sqlite3

conn = sqlite3.connect(r"C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Entregables\Code\weather.db")
cursor = conn.cursor()
path = r"C:\Users\joant\OneDrive\Stucom\MasterIA\BigData\Projecte3_Meteorologia\Datos_Proyecto"

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS cloudiness (
        cloudiness TEXT NOT NULL,
        cloudiness_id INTEGER PRIMARY KEY
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS dates (
        date TEXT NOT NULL,
        date_id INTEGER PRIMARY KEY
    )

""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS weather (
        weather TEXT NOT NULL,
        weather_id INTEGER PRIMARY KEY
    )

""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS seasons (
        estacion TEXT NOT NULL,
        estacion_id INTEGER PRIMARY KEY
    )

""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS observations (
        date_id INTEGER NOT NULL,
        precipitation REAL,
        temp_max REAL,
        temp_min REAL,
        wind REAL,
        humidity REAL NOT NULL,
        pressure REAL NOT NULL,
        solar_radiation REAL NOT NULL,
        visibility REAL NOT NULL,
        weather_id INTEGER NOT NULL,
        cloudiness_id INTEGER NOT NULL,
        estacion_id INTEGER NOT NULL,
        PRIMARY KEY (date_id, weather_id, cloudiness_id, estacion_id),
        FOREIGN KEY (date_id) REFERENCES dates(date_id),
        FOREIGN KEY (weather_id) REFERENCES weather(weather_id),
        FOREIGN KEY (cloudiness_id) REFERENCES cloudiness(cloudiness_id),
        FOREIGN KEY (estacion_id) REFERENCES seasons(estacion_id)
    )
""")

INSERTAR DATOS

In [ ]:
for csv in os.listdir(path):
    df = pd.read_csv(os.path.join(path, csv)) 
    df.to_sql(csv.split(".")[0], conn, if_exists='replace', index=False)

conn.commit()

In [ ]:
import os

def connectar_BDD():
    ruta_bd = f"{os.getcwd()}/weather.db"
    try:
        connexio = sqlite3.connect(ruta_bd)
        cursor = connexio.cursor()
    except Exception as e:
        return e
    return connexio, cursor

def guardar_tancar_BDD(connexio):
    try:
        connexio.commit()
        connexio.close()
        return True
    except Exception as e:
        print(e)
        return False

connexio, cursor = connectar_BDD()

cursor.execute("""CREATE VIEW IF NOT EXISTS full AS
SELECT
    o.date_id,
    o.precipitation,
    o.temp_max,
    o.temp_min,
    o.wind,
    o.humidity,
    o.pressure,
    o.solar_radiation,
    o.visibility,
    w.weather,
    c.cloudiness,
    s.estacion,
    d.date
FROM observations o
JOIN weather w ON o.weather_id = w.weather_id
JOIN cloudiness c ON o.cloudiness_id = c.cloudiness_id
JOIN seasons s ON o.estacion_id = s.estacion_id
JOIN dates d ON o.date_id = d.date_id
""")

query = str(input("Weather>> "))
while query != "q":
    if query == "cls":
        os.system("cls")
    else:
        try:
            cursor.execute(query)
            for line in cursor.fetchall():
                print(' | '.join(str(value) for value in line))
        except Exception as e:
            print(e)
    query = str(input("Weather>> "))
guardar_tancar_BDD(connexio)